In [117]:
%matplotlib inline
import xarray as xr
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 

import intake
import xesmf as xe
from time import gmtime, strftime

In [118]:
!pip install cf_xarray

In [119]:
import cf_xarray as cfxr

load example domain file (from RASM, but it's essentially the same as CESM - RASM has the same structure as CESM, just different component models)

In [120]:
domain_rasm = xr.open_dataset('./domain.lnd.wr50a_ar9v4.100920.nc')

define global grid using `xesmf`

In [121]:
grid_global = xe.util.grid_global(1, 1)
lon_name = 'lon'
lat_name = 'lat'

create domain file and add attributes 

In [122]:
domain = grid_global.rename({"x": lon_name, "y": lat_name})

domain[lat_name] = np.unique(domain[lat_name].values)
domain[lon_name] = np.unique(domain[lon_name].values)
domain['lon_b'] = np.unique(domain['lon_b'].values)
domain['lat_b'] = np.unique(domain['lat_b'].values)

In [123]:
attrs = {'title': 'Rhodium/CIL bias correction and downscaling 1-degree grid', 
         'history': 'created by Diana Gergel, %s' %strftime("%Y-%m-%d %H:%M:%S", gmtime()), 
         'source code': 'grid specified by xesmf, xe.util.grid_global(1, 1)'}
domain.attrs.update(attrs)
domain['lon'].attrs.update({'long_name': 'longitude of grid cell center', 'units': 'degrees_east'})
domain['lat'].attrs.update({'long_name': 'latitude of grid cell center', 'units': 'degrees_north'})
domain['lon_b'].attrs.update({'long_name': 'longitude bounds', 'units': 'degrees_east'})
domain['lat_b'].attrs.update({'long_name': 'latitude bounds', 'units': 'degrees_north'})

In [124]:
domain

<xarray.Dataset>
Dimensions:  (lat: 180, lat_b: 181, lon: 360, lon_b: 361)
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon_b    (lon_b) int64 -180 -179 -178 -177 -176 -175 ... 176 177 178 179 180
  * lat_b    (lat_b) int64 -90 -89 -88 -87 -86 -85 -84 ... 84 85 86 87 88 89 90
Data variables:
    *empty*
Attributes:
    title:        Rhodium/CIL bias correction and downscaling 1-degree grid
    history:      created by Diana Gergel, 2021-04-05 18:10:30
    source code:  grid specified by xesmf, xe.util.grid_global(1, 1)

In [125]:
domain_filename = '/home/jovyan/downscaling/downscale/domain.1x1.nc'

In [126]:
domain.to_netcdf(domain_filename)

test regridding and saved domain file to ensure that regridder service updates in dodola will address rechunking service issues 

In [127]:
ds_domainfile = xr.open_dataset(domain_filename)

In [128]:
# regrid an ERA-5 file to the domain file grid 
era5 = xr.open_dataset(os.path.join('/gcs/impactlab-data/climate/source_data/ERA-5/day/tmax/v1.1', 'tmax_daily_2000-2000.nc'))

In [129]:
regridder = xe.Regridder(era5.rename({'latitude': 'lat', 'longitude': 'lon'}), ds_domainfile, method='bilinear')

In [130]:
era5_regridded = regridder(era5['tmax'])

/srv/conda/envs/notebook/lib/python3.8/site-packages/xesmf/frontend.py:450: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


In [131]:
era5_regridded

<xarray.DataArray 'tmax' (time: 366, lat: 180, lon: 360)>
array([[[244.26470947, 244.27644348, 244.28965759, ..., 244.22506714,
         244.23828125, 244.25003052],
        [243.3735199 , 243.36764526, 243.35884094, ..., 243.40142822,
         243.38673401, 243.38085938],
        [246.64753723, 246.7268219 , 246.78555298, ..., 246.43466187,
         246.5065918 , 246.56678772],
        ...,
        [264.94824219, 264.94677734, 264.94384766, ..., 264.9614563 ,
         264.95559692, 264.95266724],
        [265.40484619, 265.41952515, 265.41659546, ..., 265.3666687 ,
         265.37548828, 265.39016724],
        [265.80419922, 265.79830933, 265.79244995, ..., 265.82473755,
         265.81741333, 265.81152344]],

       [[244.39143372, 244.40316772, 244.41635132, ..., 244.36065674,
         244.37091064, 244.37969971],
        [244.24778748, 244.29029846, 244.33720398, ..., 244.13638306,
         244.16424561, 244.20674133],
        [247.52230835, 247.59706116, 247.65715027, ..., 247.34934998,
         247.40943909, 247.45780945],
...
        [253.32640076, 253.3427124 , 253.37234497, ..., 253.28787231,
         253.29972839, 253.31158447],
        [253.77252197, 253.77548218, 253.77252197, ..., 253.74732971,
         253.76956177, 253.77104187],
        [253.53538513, 253.52944946, 253.52500916, ..., 253.54724121,
         253.54278564, 253.53834534]],

       [[245.44424438, 245.45761108, 245.47244263, ..., 245.4041748 ,
         245.4175415 , 245.43089294],
        [246.43254089, 246.47705078, 246.49337769, ..., 246.30789185,
         246.33905029, 246.38803101],
        [248.40467834, 248.48184204, 248.53674316, ..., 248.23104858,
         248.27409363, 248.32157898],
        ...,
        [250.55932617, 250.59494019, 250.6305542 , ..., 250.51333618,
         250.53410339, 250.55487061],
        [250.6305542 , 250.60385132, 250.57565308, ..., 250.66914368,
         250.68101501, 250.65579224],
        [250.3812561 , 250.39164734, 250.39906311, ..., 250.35455322,
         250.3634491 , 250.37088013]]])
Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-31
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear